## Importing Basic Library

In [1]:
import numpy as np
import pandas as pd

## Importing and Viewing Datasets

In [2]:
train = pd.read_csv('train.csv')
train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test = pd.read_csv('test.csv')
test.head(5)

test_PassengerId = test.PassengerId

In [4]:
data = [train, test]

## Analysing Dataset

In [5]:
print(train.shape)
print(test.shape)

(891, 12)
(418, 11)


In [6]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## Dealing With Missing Data

In [8]:
from sklearn.impute import SimpleImputer

def clean_missing_data(dataset):
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    dataset['Age'] = imputer.fit_transform(dataset[['Age']])
    dataset['Fare'] = imputer.fit_transform(dataset[['Fare']])

    imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    dataset['Embarked'] = imputer.fit_transform(dataset[['Embarked']])

    return dataset

for dataset in data:
    clean_missing_data(dataset)

In [9]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [10]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

## Creating New Feature and Dropping Some Columns

In [11]:
for dataset in data:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    
    dataset.drop(["Cabin", "PassengerId", "SibSp", "Name", "Ticket"], axis=1, inplace=True)

In [12]:
train.head()

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Title
0,0,3,male,22.0,0,7.2500,S,Mr
1,1,1,female,38.0,0,71.2833,C,Mrs
2,1,3,female,26.0,0,7.9250,S,Miss
3,1,1,female,35.0,0,53.1000,S,Mrs
4,0,3,male,35.0,0,8.0500,S,Mr


## LabelEncoding Categorical columns

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for dataset in data:
    dataset["Sex"] = le.fit_transform(dataset.Sex)
    dataset["Embarked"] = le.fit_transform(dataset.Embarked)

In [14]:
train.Title.value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Mlle          2
Major         2
Col           2
Countess      1
Capt          1
Ms            1
Sir           1
Lady          1
Mme           1
Don           1
Jonkheer      1
Name: Title, dtype: int64

## Encoding Other Categorical Columns

In [15]:
for dataset in data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [16]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train.head()

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Title
0,0,3,1,22.0,0,7.2500,2,1
1,1,1,0,38.0,0,71.2833,0,3
2,1,3,0,26.0,0,7.9250,2,2
3,1,1,0,35.0,0,53.1000,2,3
4,0,3,1,35.0,0,8.0500,2,1


## Seperating Dependent Variable

In [17]:
y = train["Survived"]
X = train.drop(columns = ["Survived"])

## Feature Scaling

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X = sc.fit_transform(X)
test = sc.transform(test)

## Splitting the Data

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

## Creating CatBoost Classifier Model

In [20]:
from catboost import CatBoostClassifier

classifier = CatBoostClassifier()
classifier.fit(X_train, y_train)

Learning rate set to 0.008911
0:	learn: 0.6864854	total: 209ms	remaining: 3m 29s
1:	learn: 0.6798908	total: 212ms	remaining: 1m 45s
2:	learn: 0.6729934	total: 214ms	remaining: 1m 11s
3:	learn: 0.6669041	total: 216ms	remaining: 53.8s
4:	learn: 0.6614765	total: 218ms	remaining: 43.4s
5:	learn: 0.6560129	total: 220ms	remaining: 36.5s
6:	learn: 0.6509000	total: 221ms	remaining: 31.3s
7:	learn: 0.6450471	total: 222ms	remaining: 27.6s
8:	learn: 0.6421629	total: 223ms	remaining: 24.5s
9:	learn: 0.6371110	total: 224ms	remaining: 22.2s
10:	learn: 0.6319740	total: 226ms	remaining: 20.3s
11:	learn: 0.6278339	total: 227ms	remaining: 18.7s
12:	learn: 0.6226149	total: 228ms	remaining: 17.3s
13:	learn: 0.6176785	total: 229ms	remaining: 16.2s
14:	learn: 0.6135848	total: 230ms	remaining: 15.1s
15:	learn: 0.6087416	total: 231ms	remaining: 14.2s
16:	learn: 0.6046855	total: 233ms	remaining: 13.5s
17:	learn: 0.6001391	total: 234ms	remaining: 12.8s
18:	learn: 0.5954569	total: 236ms	remaining: 12.2s
19:	lear

## Checking Performance of the model

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[101   5]
 [ 28  45]]


0.8156424581005587

## Using k-Fold Cross Validation for Better Performance

In [22]:
from sklearn.model_selection import cross_val_score

accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)

print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Learning rate set to 0.008515
0:	learn: 0.6863835	total: 3.63ms	remaining: 3.62s
1:	learn: 0.6800427	total: 5.16ms	remaining: 2.57s
2:	learn: 0.6737758	total: 6.73ms	remaining: 2.24s
3:	learn: 0.6687938	total: 8.49ms	remaining: 2.11s
4:	learn: 0.6637048	total: 9.99ms	remaining: 1.99s
5:	learn: 0.6586037	total: 11.4ms	remaining: 1.88s
6:	learn: 0.6532096	total: 12.6ms	remaining: 1.79s
7:	learn: 0.6487522	total: 13.3ms	remaining: 1.65s
8:	learn: 0.6432328	total: 14.5ms	remaining: 1.59s
9:	learn: 0.6383017	total: 15.6ms	remaining: 1.55s
10:	learn: 0.6328713	total: 16.9ms	remaining: 1.52s
11:	learn: 0.6279755	total: 18.3ms	remaining: 1.5s
12:	learn: 0.6229560	total: 19.5ms	remaining: 1.48s
13:	learn: 0.6182752	total: 20.7ms	remaining: 1.46s
14:	learn: 0.6148161	total: 21.4ms	remaining: 1.4s
15:	learn: 0.6096143	total: 22.6ms	remaining: 1.39s
16:	learn: 0.6053805	total: 23.8ms	remaining: 1.37s
17:	learn: 0.6009823	total: 24.9ms	remaining: 1.36s
18:	learn: 0.5986291	total: 25.5ms	remaining: 

## Making Prediction on Test Dataset

In [23]:
prediction = classifier.predict(test)
prediction

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [24]:
output = pd.DataFrame({'PassengerId': test_PassengerId, 'Survived': prediction})
output.to_csv('submission-catboost.csv', index=False)